In [46]:
from nltk import tokenize
import os
import pandas as pd
import json
import numpy as np

annots_folder = "OPP-115/consolidation/threshold-0.75-overlap-similarity/"
sample_data_folder = "sample/"
output_folder = "training_data/"

In [31]:
def parse_tags_recursively(d,prev_keys):
    selected_text = ""
    for k, v in d.items():
        if isinstance(v, dict):
            yield from parse_tags_recursively(v,prev_keys+"=>"+k)
        else:
            if k == "value":
                yield("{0}=>{1}".format(prev_keys,v))
            elif k == "selectedText":
                selected_text = "{0}".format(v)
                
    yield selected_text

def parse_tags_wrapper(d,prev_keys):
    tags = list(parse_tags_recursively(d,prev_keys))
    l = list(zip(tags[1::2],tags[::2]))
    d = dict()
    [d [t [0]].append(t [1]) if t [0] in list(d.keys()) else d.update({t [0]: [t [1]]}) for t in l]
    return d

def get_longest_list_in_dict_vals(d):
    longest = 0
    for k,v in d.items():
        if len(v) > longest:
            longest = len(v)
    
    return longest


In [28]:
### TESTING SANDBOX ###
filename = "453_barnesandnoble.com.csv"
df_test = pd.read_csv(annots_folder+filename, names=["annot_id","batch_id","annotator_id","policy_id","segment_id","category","tags","url","date"])
df_test.set_index("annot_id",inplace=True)
df_test.drop(['batch_id','annotator_id','url','date'],inplace=True,axis=1)
        
i = 13
tag_dict = json.loads(df_test['tags'][i])
tag_cat = df_test['category'][i]

d = parse_tags_wrapper(tag_dict,tag_cat)
del d['']

#print(json.dumps(tag_dict,indent=1))
#print("\n")
#print(json.dumps(d,indent=1))

df = pd.DataFrame.from_dict(d,orient='index')#,columns=[''])
df
#df_ohe = pd.get_dummies(df)
#df_ohe.columns = df_ohe.columns.str.lstrip("_")
#df_ohe

[('By interacting with Barnes & Noble in the manner described in this Privacy Policy at any time, you are accepting the practices described in this Privacy Policy and you consent to the application of this Privacy Policy to the collection, storage, use and disclosure of all your personal and other information as described.', 'User Choice/Control=>Choice Type=>Dont use service/feature'), ('disclosure of all your personal', 'User Choice/Control=>Choice Scope=>Third party sharing/collection'), ('', 'User Choice/Control=>User Type=>Unspecified'), ('', 'User Choice/Control=>Purpose=>Unspecified'), ('personal and other information', 'User Choice/Control=>Personal Information Type=>Generic personal information')]
{'By interacting with Barnes & Noble in the manner described in this Privacy Policy at any time, you are accepting the practices described in this Privacy Policy and you consent to the application of this Privacy Policy to the collection, storage, use and disclosure of all your perso

,0
"By interacting with Barnes & Noble in the manner described in this Privacy Policy at any time, you are accepting the practices described in this Privacy Policy and you consent to the application of this Privacy Policy to the collection, storage, use and disclosure of all your personal and other information as described.",User Choice/Control=>Choice Type=>Dont use ser...
disclosure of all your personal,User Choice/Control=>Choice Scope=>Third party...
personal and other information,User Choice/Control=>Personal Information Type...


In [40]:
## GENERATE TRAINING DATA ##

master_df = pd.DataFrame()

for filename in os.listdir(annots_folder):
    if filename.endswith(".csv"):           
        df_in = pd.read_csv(annots_folder+filename, names=["annot_id","batch_id","annotator_id","policy_id","segment_id","category","tags","url","date"])
        df_in.set_index("annot_id",inplace=True)
        df_in.drop(['batch_id','annotator_id','url','date'],inplace=True,axis=1)
        
        rows = {}
        for index, row in df_in.iterrows():
            rows.update(parse_tags_wrapper(json.loads(row['tags']),row['category']))
        
        try:
            del rows['null']
        except:
            pass
        try:
            del rows['Not selected']
        except:
            pass
        
        df_temp = pd.DataFrame.from_dict(rows,orient='index',columns=['']*get_longest_list_in_dict_vals(rows))
        master_df = pd.concat([master_df,df_temp],axis=1,sort=True)

df_ohe = pd.get_dummies(master_df)
df_ohe.columns = df_ohe.columns.str.lstrip("_")
df_ohe = df_ohe.groupby(df_ohe.columns, axis=1).agg(np.max)

In [45]:
df_ohe.to_csv("spans_and_labels_OHE.csv")

In [79]:
## GENERATE PREDICTION DATA ##
all_tokens = []
num_failed = 0

for filename in os.listdir(sample_data_folder):
    try:
        if filename.endswith(".txt"):
            text = open(sample_data_folder+filename).read()
            tokens = tokenize.sent_tokenize(text)
            all_tokens.extend(tokens)
            #print("Processed " + filename)
    except:
        num_failed += 1
        
print("Failed: " + str(num_failed))
all_tokens = list(set(all_tokens))

#p = "Good morning Dr. Adams. The patient is waiting for you in room number 3."
#tokenize.sent_tokenize(p)

Failed: 3663


In [82]:
token_list = [s.replace('\n', ' ').replace('\r', '').strip().encode('ascii',errors='ignore').decode() for s in all_tokens]

In [84]:
with open('sample_tokens.txt', 'w') as f:
    for item in token_list:
        f.write("%s\n" % item)

In [83]:
type(token_list[3])
#str(all_tokens[3])

str